## Chapter 3: Categorical Data

### 3.1 Introduction

#### 3.1.1 Tabulating and plottingcategorical data

__Example 3.2 (Rolling a die).__
As a simple example, suppose we wish to collect several rolls of a die. We
observe seven rolls of the die and save the rolls in the vector y

In [3]:
y = c(1, 4, 3, 5, 4, 2, 4)
possible.rolls = c(1, 2, 3, 4, 5, 6)
labels.rolls = c("one", "two", "three", "four", "five", "six")
fy = factor(y, levels=possible.rolls, labels=labels.rolls)
fy

[1] one   four  three five  four  two   four 
Levels: one two three four five six

In [16]:
table(fy)

fy
  one   two three  four  five   six 
    1     1     1     3     1     0 

### 3.2 Chi-square goodness of fit

 Weldon considered five or six dots showing among the 12
dice to be “successes” and other results “failures.” If a single die is fair, then
each face is equally likely to occur, so that the probability of a success (five
or six) is 1/3. The total number of successes among 12 fair dice is a binomial
random variable with success probability 1/3. The binomial probabilities are
given by the dbinom function:

In [6]:
k = 0:12
p = dbinom(k, size = 12, prob = 1/3)

In 26,306 rolls of 12 fair dice, the expected outcomes (rounded to the nearest
integer) would be:

In [7]:
Binom = round(26306*p)
names(Binom) = k

Labels for the binomial counts were applied with the names function. The
“Weldon’s Dice” data are entered below

In [8]:
Weldon = c(185, 1149, 3265, 5475, 6114, 5194, 3067, 1331, 403, 105, 14, 4, 0)
names(Weldon) = k

The binomial counts, data, and deviations between the binomial and observed counts can be summarized for display several ways; here we use the
data.frame function. To combine data in a data frame we simply list the data
vectors separated by commas. Here we also assigned a name “Diff” for the
difference between the observed (Weldon) and expected (binomial) counts.

In [9]:
data.frame(Binom, Weldon, Diff=Weldon - Binom)

,Binom,Weldon,Diff
0,203,185,-18
1,1216,1149,-67
2,3345,3265,-80
3,5576,5475,-101
4,6273,6114,-159
5,5018,5194,176
6,2927,3067,140
7,1255,1331,76
8,392,403,11
9,87,105,18


In [10]:
counts = cbind(Binom, Weldon)
barplot(counts, beside=TRUE, main = "Figure 3.2")

The barplots in Figure 3.2 appear to agree approximately. For a comparison of observed and fitted data, however, it is somewhat easier to interpret
the data on a single plot. Another possible plot for comparing the observed
and expected counts is produced by:

In [11]:
plot(k, Binom, type="h", lwd=2, lty=1, ylab="Count")
lines(k + .2, Weldon, type="h", lwd=2, lty=2)
legend(8, 5000, legend=c("Binomial", "Weldon"), lty=c(1,2), lwd=c(2,2))

A chi-square goodness-of-fit test can be applied to test whether the data
are consistent with the ‘fair dice’ binomial model. However, as we learn in
basic statistics, the expected cell counts should be large — say, at least 5 for
all cells. For a chi-square test we should collapse the categories corresponding
to 10, 11, and 12 successes into a single category. This is accomplished with
the following code.

In [12]:
cWeldon = c(Weldon[1:10], sum(Weldon[11:13]))
cWeldon

0    1    2    3    4    5    6    7    8    9      
 185 1149 3265 5475 6114 5194 3067 1331  403  105   18

In [13]:
probs = c(p[1:10], 1 - sum(p[1:10])) 
chisq.test(cWeldon, p=probs)


	Chi-squared test for given probabilities

data:  cWeldon
X-squared = 35.494, df = 10, p-value = 0.0001028


In [14]:
test = chisq.test(cWeldon, p=probs)
plot(0:10, test$residuals, xlab="k", ylab="Residual")
abline(h=0)

#### Graph of Chi-square for various k

In [33]:
x <- seq(0,20,length=100)
plot(x,dchisq(x,df=1),type="l",xlim=c(0,15),ylim=c(0,0.5),ylab="density")
lines(x,dchisq(x,df=5),lty=2)
lines(x,dchisq(x,df=10),lty=3)
abline(h=0,col="gray")
abline(v=0,col="gray")
legend("topright",legend=c("df=1","df=5","df=10"),lty=1:3)

### 3.3 Relating Two Categorical Variables
#### 3.3.1 Introduction

In [34]:
twn = read.table("twins.dat.txt", header=TRUE, sep=",", na.strings=".")

There were 183 pairs of twins who were interviewed in this study. In each pair
of twins, one twin was randomly assigned to “twin 1” and the other is called
“twin 2.” The variables EDUCL and EDUCH give the self-reported education (in
years) for twin 1 and twin 2. (In the variable definition, the last letter of “L”
refers to twin 1 and “H” refers to twin 2.) We can obtain frequency tables of
the education years of the twins by two applications of the table function.

In [35]:
table(twn$EDUCL)


 8 10 11 12 13 14 15 16 17 18 19 20 
 1  4  1 61 21 30 11 37  1 10  3  3 

In [36]:
table(twn$EDUCH)


 8  9 10 11 12 13 14 15 16 17 18 19 20 
 2  1  2  1 65 22 22 15 33  2 11  2  5 

Since there are so many educational year values, it is useful to categorize
this variable into a smaller number of more meaningful levels. __Suppose we
say that a person’s educational level is “high school” if he/she has 12 years of
education, “some college” if the years are between 13 and 15, “college degree”
for 16 years, and “graduate school” if the years are greater than 16.__ <br> The __cut__
function is very helpful for creating these new categories. In cut, the first
argument is the variable to be changed, the argument breaks is a vector
defining the breakpoints for the categories, and the argument labels is a
character vector with the labels for the new categories. We use this function
twice, once for twin 1’s educational years and again for twin 2’s educational
years.

In [66]:
c.EDUCL = cut(twn$EDUCL, breaks=c(0, 12, 15, 16, 24), labels=c("High School", "Some College", "College Degree", "Graduate School"))
cat("First Twin")
cat("Frequency Table")
table(c.EDUCL)
cat("Proportion Table")
prop.table(table(c.EDUCL))
cat("Bar Plot")
barplot(prop.table(table(c.EDUCL)), main = "Education Level of Twin 1")

First TwinFrequency Table

c.EDUCL
    High School    Some College  College Degree Graduate School 
             67              62              37              17 

Proportion Table

c.EDUCL
    High School    Some College  College Degree Graduate School 
     0.36612022      0.33879781      0.20218579      0.09289617 

Bar Plot

In [65]:
c.EDUCH = cut(twn$EDUCH, breaks=c(0, 12, 15, 16, 24), labels=c("High School", "Some College", "College Degree", "Graduate School"))
cat("Second Twin")
cat("Frequency Table")
table(c.EDUCH)
cat("Proportion Table")
prop.table(table(c.EDUCH))
cat("Bar Plot")
barplot(prop.table(table(c.EDUCH)), main = "Education Level of Twin 2")

Second TwinFrequency Table

c.EDUCH
    High School    Some College  College Degree Graduate School 
             71              59              33              20 

Proportion Table

c.EDUCH
    High School    Some College  College Degree Graduate School 
      0.3879781       0.3224044       0.1803279       0.1092896 

Bar Plot

#### 3.3.3 Contingency tables
The exploration of years of schooling in the previous section raises an interesting question. Is the educational level of twin 1 related to the educational level
for twin 2? One can answer this question by constructing a contingency table
of the educational levels for the two twins. This is easily constructed by the
table function with the two variables c.EDUCL and c.EDUCH as arguments.

In [54]:
table(c.EDUCL, c.EDUCH)

                 c.EDUCH
c.EDUCL           High School Some College College Degree Graduate School
  High School              47           16              2               2
  Some College             18           32              8               4
  College Degree            5           10             18               4
  Graduate School           1            1              5              10

Note from the contingency table that there are large counts along the
diagonal of the table where the twins have the same self-reported educational
levels. What proportion of twins have the same level? We store the table in
the variable T1 and extract the diagonal counts using the diag function

In [57]:
T1=table(c.EDUCL, c.EDUCH)
diag(T1)

High School    Some College  College Degree Graduate School 
             47              32              18              10

We can compute the proportion of “same educational level” twins by two
applications of the sum function, one on the vector of diagonal elements, and
a second on the entire table.

In [58]:
 sum(diag(T1)) / sum(T1)

[1] 0.5846995

We see that about 58% of the twins have the same educational level.

In [64]:
plot(T1)


### 3.4 Association Patterns in Contingency Tables
#### 3.4.1 Constructing a contingency table

The purpose of the twins study was to explore the relationship of educational
level with salary. We do some initial exploration by focusing on the data for
the first twin. The variable HRWAGEL contains the hourly wage (in dollars). If
one graphs the hourly wages by say, a histogram, one will observe the shape
of the wages is right-skewed. We divide the wages into four groups using the
cut function with break points 0, 7, 13, 20, and 150. We chose these break
points so we would have roughly the same number of twins in each class
interval. We assign the categorized wage to the variable c.wage.

In [68]:
c.wage = cut(twn$HRWAGEL, c(0, 7, 13, 20, 150))
table(c.wage)

c.wage
   (0,7]   (7,13]  (13,20] (20,150] 
      47       58       38       19 

In [71]:
T2 = table(c.EDUCL, c.wage)
T2

                 c.wage
c.EDUCL           (0,7] (7,13] (13,20] (20,150]
  High School        23     21      10        1
  Some College       15     23      12        5
  College Degree      7     12      14        3
  Graduate School     2      2       2       10

In [73]:
round(prop.table(T2, margin=1), digit =4)

                 c.wage
c.EDUCL            (0,7] (7,13] (13,20] (20,150]
  High School     0.4182 0.3818  0.1818   0.0182
  Some College    0.2727 0.4182  0.2182   0.0909
  College Degree  0.1944 0.3333  0.3889   0.0833
  Graduate School 0.1250 0.1250  0.1250   0.6250

In [74]:
P = prop.table(T2, margin = 1)
barplot(t(P), ylim=c(0, 1.3), ylab="PROPORTION", legend.text=dimnames(P)$c.wage, args.legend=list(x = "top"))

In [75]:
barplot(t(P), beside=T, legend.text=dimnames(P)$c.wage, args.legend=list(x="topleft"), ylab="PROPORTION")

### 3.5 Testing Independence by a Chi-square Test


In [77]:
T2 = table(c.EDUCL, c.wage)
S = chisq.test(T2)
print(S)

Warning message in chisq.test(T2):
“Chi-squared approximation may be incorrect”


	Pearson's Chi-squared test

data:  T2
X-squared = 54.578, df = 9, p-value = 1.466e-08



In [78]:
S$expected

,"(0,7]","(7,13]","(13,20]","(20,150]"
High School,15.956790,19.691358,12.901235,6.450617
Some College,15.956790,19.691358,12.901235,6.450617
College Degree,10.444444,12.888889,8.444444,4.222222
Graduate School,4.641975,5.728395,3.753086,1.876543


In [80]:
sum((T2 - S$expected)^2 / S$expected) #X-squared

[1] 54.57759

In [81]:
1 - pchisq(54.57759, df=9) #p-value

[1] 1.465839e-08

In [82]:
names(S)

[1] "statistic" "parameter" "p.value"   "method"    "data.name" "observed" 
[7] "expected"  "residuals" "stdres"

In [83]:
S$residuals

                 c.wage
c.EDUCL                (0,7]     (7,13]    (13,20]   (20,150]
  High School      1.7631849  0.2949056 -0.8077318 -2.1460758
  Some College    -0.2395212  0.7456104 -0.2509124 -0.5711527
  College Degree  -1.0658020 -0.2475938  1.9117978 -0.5948119
  Graduate School -1.2262453 -1.5577776 -0.9049176  5.9300942

In [84]:
mosaicplot(T2, shade=TRUE)